In [2]:
import numpy as np
import pickle

ModuleNotFoundError: No module named 'numpy'

In [ ]:
BOARD_ROWS = 3
BOARD_COLS = 3

In [ ]:
class State:
    def __init__(self, p1, p2):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.p1 = p1
        self.p2 = p2
        self.isEnd = False
        self.boardHash = None
        # init p1 plays first
        self.playerSymbol = 1
        
    def getHash(self):
        self.boardHash = str(self.board.reshape(BOARD_COLS * BOARD_ROWS))
        return self.boardHash
    
    def winner(self):
        for i in range(BOARD_ROWS):
            if sum(self.board[i, :]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[i, :]) == -3:
                self.isEnd = True
                return -1
            
        for i in range(BOARD_COLS):
            if sum(self.board[:, i]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[:, i]) == -3:
                self.isEnd = True
                return -1
            
        diag_sum1 = sum([self.board[i, i] for i in range(BOARD_COLS)])
        diag_sum2 = sum([self.board[i, BOARD_COLS - i - 1] for i in range(BOARD_COLS)])
        diag_sum = max(diag_sum1, diag_sum2)
        if diag_sum == 3:
            self.isEnd = True
            return 1
        if diag_sum == -3:
            self.isEnd = True
            return -1
        
        if len(self.availablePositions()) == 0:
            self.isEnd = True
            return 0
        
        self.isEnd = False
        return None
    
    def availablePositions(self):
        positions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == 0:
                    positions.append((i, j))
        return positions
    
    def updateState(self, position):
        self.board[position] = self.playerSymbol
        self.playerSymbol = -1 if self.playerSymbol == 1 else 1
        
    def giveReward(self):
        result = self.winner()
        if result == 1:
            self.p1.feedReward(1)
            self.p2.feedReward(0)
        elif result == -1:
            self.p1.feedReward(0)
            self.p2.feedReward(1)
        else:
            self.p1.feedReward(0.1)
            self.p2.feedReward(0.5)
            
    def reset(self):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.boardHash = None
        self.isEnd = False
        self.playerSymbol = 1
        
    def play(self, rounds=100):
        for i in range(rounds):
            if i % 1000 == 0:
                print("Rounds {}".format(i))
            while not self.isEnd:
                positions = self.availablePositions()
                p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
                self.updateState(p1_action)
                board_hash = self.getHash()
                self.p1.addState(board_hash)
                
                win = self.winner()
                if win is not None:
                    self.giveReward()
                    self.p1.reset()
                    self.p2.reset()
                    self.reset()
                    break
                    
                else:
                    positions = self.availablePositions()
                    p2_action = self.p2.chooseAction(positions, self.board, self.playerSymbol)
                    self.updateState(p2_action)
                    board_hash = self.getHash()
                    self.p2.addState(board_hash)
                    
                    win = self.winner()
                    if win is not None:
                        self.giveReward()
                        self.p1.reset()
                        self.p2.reset()
                        self.reset()
                        break
                        
    def play2(self):
        while not self.isEnd:
            positions = self.availablePositions()
            p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
            self.updateState(p1_action)
            self.showBoard()
            win = self.winner()
            if win is not None:
                if win == 1:
                    print(self.p1.name, "wins!")
                else:
                    print("It's a tie!")
                self.reset()
                break
                
            positions = self.availablePositions()
            p2_action = self.p2.chooseAction(positions, self.board, self.playerSymbol)
            
            self.updateState(p2_action)
            self.showBoard()
            win = self.winner()
            if win is not None:
                if win == -1:
                    print(self.p2.name, "wins!")
                else:
                    print("It's a tie!")
                self.reset()
                break
                
    def showBoard(self):
        for i in range(0, BOARD_ROWS):
            print('-------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'x'
                if self.board[i, j] == -1:
                    token = 'o'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' |'
            print(out)
        print('-------------')




In [ ]:


class Player:
    def __init__(self, name, exp_rate=0.3):
        self.name = name
        self.states = []
        self.lr = 0.2
        self.exp_rate = exp_rate
        self.decay_gamma = 0.9
        self.states_values = {}

    def getHash(self, board):
        return str(board.reshape(BOARD_ROWS * BOARD_COLS))

    def chooseAction(self, positions, current_board, symbol):
        if np.random.uniform(0, 1) <= self.exp_rate:
            idx = np.random.choice(len(positions))
            action = positions[idx]
        else:
            value_max = -999
            for p in positions:
                next_board = current_board.copy()
                next_board[p] = symbol
                next_board_hash = self.getHash(next_board)
                if self.states_values.get(next_board_hash) is None:
                    value = 0
                else:
                    value = self.states_values.get(next_board_hash)
                if value >= value_max:
                    value_max = value
                    action = p
        return action

    def addState(self, state):
        self.states.append(state)

    def feedReward(self, reward):
        for state in reversed(self.states):
            if self.states_values.get(state) is None:
                self.states_values[state] = 0
            self.states_values[state] += self.lr * (self.decay_gamma * reward - self.states_values[state])
            reward = self.states_values[state]

    def reset(self):
        self.states = []

    def savePolicy(self):
        with open('policy_' + str(self.name), 'wb') as fw:
            pickle.dump(self.states_values, fw)

    def loadPolicy(self, file):
        with open(file, 'rb') as fr:
            self.states_values = pickle.load(fr)

# Example usage:
# Save policy to a file
# p1.savePolicy()
# Load policy from a file
# p1.loadPolicy("policy_Player1")


In [ ]:

        
class HumanPlayer:
    def __init__(self, name):
        self.name = name

    def chooseAction(self, positions, current_board, symbol):
        while True:
            try:
                row = int(input("Enter the row (0, 1, 2): "))
                col = int(input("Enter the column (0, 1, 2): "))
                action = (row, col)

                if action in positions:
                    return action
                else:
                    print("Invalid move. Please try again.")
            except ValueError:
                print("Invalid input. Please enter row and column as integers.")

    def addState(self, state):
        pass

    def feedReward(self, reward):
        pass

    def reset(self):
        pass

# Example usage:
# human_player = HumanPlayer(name="Human")
# Use this player to play the game by taking input from a human player.


In [ ]:
p1 = Player("p1")
p2 = Player("p2")

st = State(p1, p2)
print("training...")
st.play(10000)

training...
Rounds 0
Rounds 1000
Rounds 2000
Rounds 3000
Rounds 4000
Rounds 5000
Rounds 6000
Rounds 7000
Rounds 8000
Rounds 9000


In [ ]:
p1.savePolicy()
p2.savePolicy()

In [ ]:
p1.loadPolicy("policy_p1")

In [ ]:
p1 = Player("computer", exp_rate=0)
p1.loadPolicy("policy_p1")

p2 = HumanPlayer("human")

st = State(p1, p2)
st.play2()

-------------
|   |  |  |
-------------
|   |  |  |
-------------
| x |  |  |
-------------
Enter the row (0, 1, 2): 1
Enter the column (0, 1, 2): 2
-------------
|   |  |  |
-------------
|   |  |o |
-------------
| x |  |  |
-------------
-------------
|   |  |  |
-------------
|   |  |o |
-------------
| x |  |x |
-------------
Enter the row (0, 1, 2): 2
Enter the column (0, 1, 2): 2
Invalid move. Please try again.
Enter the row (0, 1, 2): 2
Enter the column (0, 1, 2): 2
Invalid move. Please try again.
Enter the row (0, 1, 2): 2
Enter the column (0, 1, 2): 1
-------------
|   |  |  |
-------------
|   |  |o |
-------------
| x |o |x |
-------------
-------------
|   |  |  |
-------------
|   |x |o |
-------------
| x |o |x |
-------------
Enter the row (0, 1, 2): 2
Enter the column (0, 1, 2): 1
Invalid move. Please try again.
Enter the row (0, 1, 2): 2
Enter the column (0, 1, 2): 0
Invalid move. Please try again.
Enter the row (0, 1, 2): 2
Enter the column (0, 1, 2): 1
Invalid move.